In [1]:
# The intent of the notebook is to begin modelling the baseball data. This will likely not be the final model, 
#     however,z it will act as a building block to more advanced models.
# Version: 1.0

In [25]:
import import_ipynb
from sklearn import svm
from pymysql import connect
from sys import path
from pathlib import Path
path.append('../../../') 
from BaseballAnalytics.bin.app_utils.queries import Queries
from BaseballAnalytics.bin.app_utils.common_help import Log_Helper
from pickle import dump, load

In [3]:
# Connect to the database.
conn = connect(host="localhost", user="root", passwd="praquplDop#odlg73h?c", db="baseball_stats_db")
qu = Queries(conn)

In [4]:
# Get all the game ids. The information is returned as 
#    (Game_ID, year, day, month, Home_Score, Vis_Score, Home_Team, Visitng_Team, Home_Win) where a winning home team is flagged at 0.
game_outcomes = qu.get_game_outcomes()         
game_outcomes[0:5]

(('BOS199004090', 1990, 9, 4, 5, 2, 'BOS', 'DET', 0),
 ('CAL199004090', 1990, 9, 4, 4, 7, 'CAL', 'SEA', 1),
 ('CHA199004090', 1990, 9, 4, 2, 1, 'CHA', 'MIL', 0),
 ('HOU199004090', 1990, 9, 4, 4, 8, 'HOU', 'CIN', 1),
 ('KCA199004090', 1990, 9, 4, 6, 7, 'KCA', 'BAL', 1))

In [5]:
# Filter the games past May 15 (Approximentally one week) to acquire some data.
game_outcomes_cpy = list(game_outcomes)
filter_games = []
for game in game_outcomes_cpy:
    if (game[2] < 15 and game[3] < 5):        # Remove all games prior to April 15.
        pass
    else:
        filter_games.append(game)
game_outcomes_cpy = None                      # Send the data to the ether to reserve space.
filter_games[0:10]

[('ATL199004150', 1990, 15, 4, 6, 13, 'ATL', 'CIN', 1),
 ('CAL199004150', 1990, 15, 4, 4, 1, 'CAL', 'MIN', 0),
 ('CHA199004150', 1990, 15, 4, 4, 1, 'CHA', 'CLE', 0),
 ('DET199004150', 1990, 15, 4, 6, 4, 'DET', 'BAL', 0),
 ('HOU199004150', 1990, 15, 4, 4, 5, 'HOU', 'LAN', 1),
 ('KCA199004150', 1990, 15, 4, 4, 5, 'KCA', 'TOR', 1),
 ('MON199004150', 1990, 15, 4, 3, 1, 'MON', 'NYN', 0),
 ('NYA199004150', 1990, 15, 4, 3, 1, 'NYA', 'TEX', 0),
 ('PHI199004150', 1990, 15, 4, 4, 0, 'PHI', 'SLN', 0),
 ('PIT199004150', 1990, 15, 4, 3, 3, 'PIT', 'CHN', 0)]

In [6]:
# Retrieve only the game ids.
game_ids = [game_id[0] for game_id in filter_games]
game_ids = game_ids[200:]                                   # Cut off the first 200 games.
print("The number of game ids: {}".format(len(game_ids)))
game_ids[0:5]

The number of game ids: 65734


['LAN199005020',
 'MON199005020',
 'NYA199005020',
 'NYN199005020',
 'PHI199005020']

In [7]:
# Fetch the players and pitchers from the game.
data_locs = Path(r'C:\Users\micha\Documents\Baseball_Analytics_Source_Data\model_v1')
pitchers_pickle = 'game_pitchers.pickle'
batters_pickle = 'game_players.pickle'
offensive_feat_pickle = 'raw_offensive_features.pickle'
offensive_features = qu.get_all_offensive_features(data_locs / offensive_feat_pickle)
all_pitcher = qu.get_pitchers_in_all_games_vOne(data_locs / pitchers_pickle)
all_batters = qu.get_batters_in_all_games_vOne(data_locs / batters_pickle)

In [8]:
# Gather all the features from every game.
features_pickle = 'game_features.pickle'
lh = Log_Helper()
all_features = {}
num_games = len(game_ids)
lh.print_progress_bar(0, num_games, prefix = 'Progress:', suffix = 'Complete', length = 50)    # Initial call to print 0% progress
counter = 0
for num, game_id in enumerate(game_ids):
    all_features[game_id] = qu.get_game_features(all_batters, all_pitcher, offensive_features, game_id)
    lh.print_progress_bar(num + 1, num_games, prefix = 'Progress:', suffix = 'Complete', length = 50)
    counter += 1
    if counter > 20:
        break     
with open(data_locs / features_pickle, 'wb') as f:
    dump(all_features, f)

1.0.0039322999999740205
2.0.005399899999986246
3.1.9199044999999728
4.2.3726151000000186
5.2.3736101000000076
6.2.3741196999999943
7.2.436787700000025
8.3.243981099999985
1.0.0004749000000288106
2.0.0012790000000109103
3.0.053573900000003505
4.1.195400100000029
5.1.196592800000019
6.1.1970607000000086
7.1.2098063000000252
8.1.7141745000000128
1.0.0007506999999691288
2.0.0009799999999700049
3.0.04176749999999174
4.0.47547969999999395
5.0.47633309999997664
6.0.4765279999999734
7.0.5199929999999995
8.1.5305667999999741
1.0.00035309999998389685
2.0.0005570999999804371
3.0.04399919999997337
4.0.4316054999999892
5.0.4321999000000005
6.0.4323684999999955
7.0.4645253999999568
8.1.594618999999966
1.0.00039930000002641464
2.0.001042799999993349
3.0.17026420000001963
4.0.7170691000000033
5.0.7181299999999737
6.0.7186872999999991
7.0.7464216000000192
8.1.9688436999999794
1.0.0016825999999809937
2.0.002727499999991778
3.0.046752299999980096
4.1.580089600000008
5.1.5807098000000224
6.1.5808890999999